In [17]:
import requests
import pandas as pd

In [18]:
response = requests.get('https://www.forbes.com/forbesapi/org/top-colleges/2021/position/true.json?limit=1000&fields=organizationName,academics,state,financialAid,rank,medianBaseSalary,campusSetting,studentPopulation,organization,description')

responseObject = response.json()

responseData = responseObject['organizationList']['organizationsLists']

data_main = pd.DataFrame(responseData)

In [19]:
data_main

,description,rank,organization,organizationName,state,studentPopulation,academics,financialAid,campusSetting,medianBaseSalary
0,One of the top public universities in the coun...,1,"{'naturalId': 'fred/college/64', 'name': 'Univ...","University of California, Berkeley",CA,43185,"{'type': 'Public', 'carnegieClassification': '...","{'totalGrantAid': 61548859.0, 'percentOfStuden...",Urban,138800.0
1,"The second oldest Ivy League institution, Yale...",2,"{'naturalId': 'fred/college/10', 'name': 'Yale...",Yale University,CT,13609,"{'type': 'Private', 'carnegieClassification': ...","{'totalGrantAid': 48792777.0, 'percentOfStuden...",Urban,141300.0
2,Princeton is a leading private research univer...,3,"{'naturalId': 'fred/college/2', 'name': 'Princ...",Princeton University,NJ,8419,"{'type': 'Private', 'carnegieClassification': ...","{'totalGrantAid': 43368213.0, 'percentOfStuden...",Suburban,150500.0
3,"Located just outside of Palo Alto, CA, Stanfor...",4,"{'naturalId': 'fred/college/6', 'name': 'Stanf...",Stanford University,CA,17249,"{'type': 'Private', 'carnegieClassification': ...","{'totalGrantAid': 51958388.0, 'percentOfStuden...",Suburban,147100.0
4,Columbia University in the City of New York is...,5,"{'naturalId': 'fred/college/13', 'name': 'Colu...",Columbia University,NY,31456,"{'type': 'Private', 'carnegieClassification': ...","{'totalGrantAid': 42294774.0, 'percentOfStuden...",Urban,132100.0
...,...,...,...,...,...,...,...,...,...,...
595,"Affiliated with the Presbyterian Church, Whitw...",596,"{'naturalId': 'fred/college/208', 'name': 'Whi...",Whitworth University,WA,2814,"{'type': 'Private', 'carnegieClassification': ...","{'totalGrantAid': 22087881.0, 'percentOfStuden...",Suburban,89100.0
596,"Augustana University in Sioux Falls, SD, one o...",597,"{'naturalId': 'fred/college/307', 'name': 'Aug...",Augustana University,SD,2113,"{'type': 'Private', 'carnegieClassification': ...","{'totalGrantAid': 10137731.0, 'percentOfStuden...",Urban,91900.0
597,"A small, Lutheran liberal arts college in the ...",598,"{'naturalId': 'fred/college/121', 'name': 'Lut...",Luther College,IA,1951,"{'type': 'Private', 'carnegieClassification': ...","{'totalGrantAid': 17709147.0, 'percentOfStuden...",Rural,83800.0
598,Coe College is a liberal arts school in Cedar ...,599,"{'naturalId': 'fred/college/224', 'name': 'Coe...",Coe College,IA,1430,"{'type': 'Private', 'carnegieClassification': ...","{'totalGrantAid': 14071402.0, 'percentOfStuden...",Urban,88300.0


#### Adding 'organization' to main table

In [20]:
# add organization
organization = [ x['organization'] for x in responseData]
data_main = pd.concat([data_main, pd.DataFrame(organization)], axis=1)

#### Add single 'Primary_Key' column to main table

In [21]:
# add naturalId column
data_main['Primary_Key'] = [ x['organization']['naturalId'].split('/')[-1] for x in responseData]

#### Adding 'organization/geoLocation' to main table

In [22]:

# add geoLocation columns
rows = []
for organizationsLists in responseData:
    row = {}    
    # row['Primary_Key'] = organizationsLists['organization']['naturalId'].split('/')[-1]
    try:
        row['latitude'] = organizationsLists['organization']['geoLocation']['latitude']
    except:
        pass
    try:
        row['longitude']= organizationsLists['organization']['geoLocation']['longitude']
    except:
        pass
    rows.append(row)
data_main = pd.concat([data_main, pd.DataFrame(rows)], axis=1)

#### Adding 'Academics' to main table

In [23]:
academics = [ # items in the 'academics' key to be unpacked
    'attendanceStatus',
    'firstToSecondYearRetention',
    'overallGraduationRates',
    'enrollmentByGender',
    'graduationRateByGender',
    'enrollmentByRace',
    'graduationRateByRace']

def restructure(list_of_dictionaries,name): # change the shape of the json
    row = {}
    for item in list_of_dictionaries:
        key_value = [value for key,value in item.items()]
        key_name = name + '_' + key_value[0]
        row[key_name] = key_value[1]
    return row

def dataFrame_of_Item(item): # build a data frame from all json's 
    list_of_rows = []
    for organization in responseData:
        # create reshaped json
        graduationRateByRace = restructure(organization['academics'][item],item)
        # add primary_key number
        # graduationRateByRace['PrimaryKey'] = organization['organization']['naturalId'].split('/')[-1]
        # append to list
        list_of_rows.append(graduationRateByRace)
    return pd.DataFrame(list_of_rows)

# add date frames to main data
df_ls = []

for item in academics:
    df_ls.append(dataFrame_of_Item(item))

academics_unpacked = pd.concat(df_ls, axis=1)

data_main = pd.concat([data_main,academics_unpacked], axis=1)



#### Adding 'organization/socialNetworks' to main table

In [24]:
def restructure(list_of_dictionaries): # input: list of dictionaries | output: structured dictionary
    row = {}
    for item in list_of_dictionaries:
        key_value = [value for key,value in item.items()]
        key_name =key_value[0]
        row[key_name] = key_value[1]
    return row

def dataFrame_of_Item(col_name,item,TF): # input: name of dict to unpack | output: data frame
    list_of_rows = []
    for organization in responseData:
        # create json
        dictionary = {}
        try:
            # create reshaped json
            dictionary = dictionary | restructure(organization[col_name][item])
        except:
            pass
        # add primary_key number
        if TF:
            dictionary['Primary_Key'] = organization['organization']['naturalId'].split('/')[-1]
        # append to list
        list_of_rows.append(dictionary)
    return pd.DataFrame(list_of_rows)

df_socialNetworks = dataFrame_of_Item('organization','socialNetworks',True)

data_main = data_main.merge(df_socialNetworks, on='Primary_Key')

#### Adding 'financialAid' to main table

In [25]:
financialAid = [
    'grantAidByType',
    'avgGrantAidByType',
    'loansByType',
    'avgLoansByType']

def restructure(list_of_dictionaries): # input: list of dictionaries | output: structured dictionary
    row = {}
    for item in list_of_dictionaries:
        key_value = [value for key,value in item.items()]
        key_name =key_value[0]
        row[key_name] = key_value[1]
    return row

def dataFrame_of_Item(col_name,item,TF): # input: name of dict to unpack | output: data frame
    list_of_rows = []
    for organization in responseData:
        # create json
        dictionary = {}
        try:
            # create reshaped json
            dictionary = dictionary | restructure(organization[col_name][item])
        except:
            pass
        # add primary_key number
        if TF:
            dictionary['Primary_Key'] = organization['organization']['naturalId'].split('/')[-1]
        # append to list
        list_of_rows.append(dictionary)
    return pd.DataFrame(list_of_rows)

# add date frames to main data
df_ls = []

for item in financialAid:
    df_ls.append(dataFrame_of_Item('financialAid',item,True))

financialAid_unpacked = pd.concat(df_ls, axis=1)

financialAid_unpacked = financialAid_unpacked.loc[:, ~financialAid_unpacked.columns.duplicated()]

data_main = data_main.merge(financialAid_unpacked)

data_main = data_main.loc[:, ~data_main.columns.duplicated()]

drop_columns = ['organization','academics','financialAid','listImages','geoLocation','visible','relatedVisible','imageExists','socialNetworks','collegeMedia']

data_main.drop(drop_columns, axis=1, inplace=True)

In [30]:
data_main[['latitude', 'longitude']]

,latitude,longitude
0,37.869236,-122.258393
1,41.314042,-72.923425
2,40.349855,-74.659119
3,37.431370,-122.168924
4,40.806515,-73.961288
...,...,...
595,47.750994,-117.415201
596,43.527114,-96.736267
597,43.313226,-91.799646
598,41.985318,-91.657250


# ------------------ TEST AREA ------------------

# WHAT TO DO:
* ~~unpack 'organization'~~
    * ~~unpack 'geoLocation'~~
    * ~~unpack 'socialNetworks'~~
* ~~unpack 'academics'~~
    * ~~unpack 'attendanceStatus'~~
    * ~~unpack 'firstToSecondYearRetention'~~
    * ~~unpack 'overallGraduationRates'~~
    * ~~unpack 'enrollmentByGender'~~
    * ~~unpack 'graduationRateByGender'~~
    * ~~unpack 'enrollmentByRace'~~
    * ~~unpack 'graduationRateByRace'~~
* ~~unpack 'financialAid'~~
    * ~~grantAidByType~~
    * ~~avgGrantAidByType~~
    * ~~loansByType~~
    * ~~avgLoansByType~~
* ~~drop unnecessary columns~~

In [ ]:
data_main.shape


In [ ]:
col_to_drop = [
       'organization',
       'academics',
       'financialAid',
       'listImages',
       'visible',
       'relatedVisible',
       'imageExists',
       'recentContentCount',
       'country',
       'collegeMedia',
       'landscapeImage',
       'industries',
       'embargo',
       'image',
       'industry',
       'ceoName',
       'ceoTitle',
       'premiumProfile',
       'employees',
       'portraitImage',
       'naturalId',
       'geoLocation',
       'uri',
       'uris',
       'socialNetworks',
       'placeUri'
       ]

data_main.drop(col_to_drop, axis=1, inplace=True)


In [ ]:

data_main.to_csv('University_Data.csv')

In [ ]:
pd.options.display.max_columns = None
data_main

In [ ]:
pd.options.display.max_columns = None
data_main.head()

In [ ]:
data_main[data_main.organizationName == 'Brigham Young University']

In [ ]:
data_main[data_main.organizationName.str.contains('Brigham')]

In [ ]:
pd.options.display.max_rows = 100
# data_main.sort_values(by='enrollmentByGender_enrollmentFemale', ascending=False)

In [ ]:
data_main.columns

In [40]:
pd.options.display.max_rows = None

data_main[data_main.].dropna(subset=['latitude'])

,description,rank,organizationName,state,studentPopulation,campusSetting,medianBaseSalary,naturalId,name,uri,...,YouTube,federalGrant,pellGrant,otherFederalGrant,stateLocalGrant,institutionalGrant,anyGrant,anyLoan,federalLoan,nonFederalLoan
0,One of the top public universities in the coun...,1,"University of California, Berkeley",CA,43185,Urban,138800.0,fred/college/64,"University of California, Berkeley",university-of-california-berkeley,...,http://www.berkeley.edu/tour/,23.0,27.0,13.0,31.0,51.0,19126.0,23.0,23.0,12.0
1,"The second oldest Ivy League institution, Yale...",2,Yale University,CT,13609,Urban,141300.0,fred/college/10,Yale University,yale-university,...,http://admissions.yale.edu/virtual-tour,20.0,18.0,9.0,NaN,56.0,55827.0,6.0,6.0,NaN
2,Princeton is a leading private research univer...,3,Princeton University,NJ,8419,Suburban,150500.0,fred/college/2,Princeton University,princeton-university,...,http://www.princeton.edu/~oktour/virtualtour/,21.0,19.0,4.0,2.0,62.0,52188.0,10.0,4.0,8.0
3,"Located just outside of Palo Alto, CA, Stanfor...",4,Stanford University,CA,17249,Suburban,147100.0,fred/college/6,Stanford University,stanford-university,...,NaN,18.0,17.0,8.0,5.0,56.0,54808.0,7.0,7.0,1.0
4,Columbia University in the City of New York is...,5,Columbia University,NY,31456,Urban,132100.0,fred/college/13,Columbia University,columbia-university-in-the-city-of-new-york,...,NaN,18.0,25.0,10.0,6.0,52.0,55071.0,10.0,8.0,2.0
5,"A leading global research university, MIT attr...",6,Massachusetts Institute of Technology,MA,11520,Urban,158100.0,fred/college/5,Massachusetts Institute of Technology,massachusetts-institute-of-technology,...,http://web.mit.edu/vrtour/,19.0,18.0,16.0,2.0,60.0,49775.0,9.0,8.0,2.0
6,"Founded in 1636 in Cambridge, Massachusetts, H...",7,Harvard University,MA,31655,Urban,147700.0,fred/college/8,Harvard University,harvard-university,...,NaN,18.0,12.0,12.0,3.0,57.0,55455.0,7.0,2.0,6.0
7,"Applicants to the University of California, Lo...",8,"University of California, Los Angeles",CA,44371,Urban,122400.0,fred/college/70,"University of California, Los Angeles",university-of-california-los-angeles,...,virtualtour.ucla.edu,26.0,32.0,3.0,33.0,46.0,19595.0,28.0,27.0,7.0
8,The University of Pennsylvania covers almost 3...,9,University of Pennsylvania,PA,26675,Urban,142900.0,fred/college/36,University of Pennsylvania,university-of-pennsylvania,...,NaN,13.0,13.0,9.0,5.0,49.0,50532.0,13.0,12.0,3.0
9,"An elite private university in Evanston, IL, N...",10,Northwestern University,IL,22448,Urban,120300.0,fred/college/18,Northwestern University,northwestern-university,...,NaN,21.0,18.0,20.0,8.0,57.0,48438.0,26.0,25.0,1.0


In [42]:
data_main.state.unique()

array(['CA', 'CT', 'NJ', 'NY', 'MA', 'PA', 'IL', 'NH', 'NC', 'TN', 'DC',
       'MI', 'TX', 'FL', 'RI', 'WA', 'VA', 'MO', 'GA', 'ME', 'MD', 'IN',
       'VT', 'UT', 'WI', 'CO', 'MN', 'OH', 'SC', 'DE', 'IA', 'LA', 'AZ',
       'OK', 'OR', 'AL', 'KS', 'AR', 'NE', 'KY', 'MS', 'NV', 'WV', 'ND',
       'NM', 'MT', 'HI', 'WY', 'ID', 'SD', 'AK'], dtype=object)

In [53]:
data_main.longitude.dropna().max()

85.5016

In [56]:
data_main[data_main.longitude == 85.5016][['latitude','longitude']]

,latitude,longitude
303,36.1628,85.5016


In [57]:
data_main[data_main.longitude == 85.5016]

,description,rank,organizationName,state,studentPopulation,campusSetting,medianBaseSalary,naturalId,name,uri,...,YouTube,federalGrant,pellGrant,otherFederalGrant,stateLocalGrant,institutionalGrant,anyGrant,anyLoan,federalLoan,nonFederalLoan
303,Tennessee Tech is a midsize public research un...,304,Tennessee Technological University,TN,10140,Rural,92000.0,fred/college/820,Tennessee Technological University,tennessee-technological-university,...,https://www.youtube.com/user/ttunews,37.0,36.0,6.0,90.0,48.0,7777.0,50.0,49.0,6.0
